In [1]:
from lapis import lapis, noaa_dav, gps_time
# import leafmap
import leafmap.leafmap as leafmap  # for ipyleaflet only

### Inputs: coordinates (converted to circles), polygon features (shapefiles, gpkg, geojson etc., -- Only single part), or bounding boxes

In [2]:
locations_dict = [
    {
        'name': 'gage id01650500',
        'data_type': 'coords',
        'value': (-77.0293611, 39.06552778), #longitude, latitude
        'buffer': 50, # in meters -- default value is 50 meters unless specified 
    },
    {
        'name': 'gage id 01651000',
        'data_type': 'coords',
        'value': (-76.96513889, 38.95255556), #longitude, latitude
        'buffer': 50, # in meters -- default value is 50 meters unless specified 
    },
    {
        'name': 'washington dc',
        'data_type': 'file',
        'value': 'demo/district_of_columbia_boundary.geojson'
    },
    {
        'name': 'augusta county, va',
        'data_type': 'bbox',
        'value': [-79.53330839559011, 37.88158664008918, -78.74939547673364, 38.477678774587105]
    }
]

### Find all lidar collections that intersect provided locations (using USIAEI)

USIAEI lists all lidar collections, but sometimes paths to EPT (Entwine Point Tiles) files are not included. Another source of these EPT files is NOAA's Digital Access Viewer (DAV). First step is to webscrape all of NOAA's collection names and corresponding EPT files if any and cross walk the records from USIAEI and NOAA.

In [3]:
find_lidar = lapis.SearchLidarInventory(locations_dict)
locations, collections = find_lidar.get_lidar_collections()

### Result(s)

In [4]:
locations[['name', 'geometry']]

,name,geometry
0,gage id01650500,"POLYGON ((-77.02879 39.06544, -77.02880 39.065..."
1,gage id 01651000,"POLYGON ((-76.96457 38.95247, -76.96458 38.952..."
2,washington dc,"POLYGON ((-77.00244 38.96567, -77.00150 38.964..."
3,"augusta county, va","POLYGON ((-79.53331 37.88159, -78.74940 37.881..."


In [5]:
collections.head(3)

,geometry,OBJECTID,ID,Title,DataType,Status,Links,pointspacing,verticalaccuracy,horizontalaccuracy,...,horizontaldatum,restrictions,leafOnOff,AlternateTitle,notes,RecordOwner,ContractSpec,Shape_Length,Shape_Area,name
0,"POLYGON ((-77.16494 39.30900, -77.16678 39.312...",161,28493,2018 MNCPPC Lidar: Montgomery and Prince Georg...,Lidar-Topo,Complete,"{""links"":[{""link"":""https:\/\/lidar.geodata.md....",0.7 m (spec),"{""vertAccuracy"":[{""VertDescription"":""4.6 cm RM...",0.5 m,...,NAD83 HARN,Public,None,None,Plans are to submit to USGS after acceptance.,USGS,USGS LBS 1.2,441920.168052,4.317304e+09,gage id01650500
1,"POLYGON ((-77.45889 39.21983, -77.47362 39.208...",876,24506,2008 Montgomery County Lidar,Lidar-Topo,Complete,"{""links"":[{""link"":""http:\/\/www6.montgomerycou...",Not Provided,"{""vertAccuracy"":[{""VertDescription"":""15 cm RMS...",1 m,...,NAD83,Other,None,None,,USGS,Unknown,221628.863882,2.181791e+09,gage id01650500
2,"POLYGON ((-77.11908 39.27012, -77.12978 39.271...",1556,42214,2021 MNCPPC Lidar: Montgomery and Prince Georg...,Lidar-Topo,Complete,None,0.35 m (spec),"{""vertAccuracy"":[{""VertDescription"":""10 cm (sp...",0.5 m,...,NAD83 HARN,Public,None,USGS: MD_2020MontgomeryPrinceGeorges_C22,None,USGS,Unknown,426673.478813,4.243600e+09,gage id01650500


In [6]:
print(f"total collections: {collections.shape[0]}\n")

total collections: 28



### Cross-reference NOAA DAV - by web scraping 

In [7]:
noaa_dav_gdf = noaa_dav.scrape_digital_coast_repo()

### Parse lidar collections and extract EPT (if any)

In [8]:
collections = lapis.ProcessLidarCollections(collections, locations, noaa_dav_gdf).execute()

Curl failure: Timeout was reached


Notable columns in this dataframe:
- `Title`: Lidar collection title
- `collectionyear`: from USIEI -- year of lidar collection
- `ept_usiaei`: EPT file path from USIAEI if present
- `noaa_id`: If a USIEI collection is present in NOAA DAV repo then the unique will be present
- `ID #`: same as `noaa_id` for cross-walk
- `ept_noaa`: EPT file paths pulled from NOAA DAV
- `ept`: reconciled EPT urls -- this cross-references USIAEI and NOAA DAV and used to retrieve point clouds
- `ept_crs`: CRS for EPT files which is needed for on-the-fly transformation
- `ept_gdf`: GeoDataframe of the original location feature reprojected to the native CRS of the EPT file

In [9]:
collections[['name', 'Title', 'collectionyear', 'collectiondate', 'meets3dep', 'Status']]

,name,Title,collectionyear,collectiondate,meets3dep,Status
0,gage id01650500,2018 MNCPPC Lidar: Montgomery and Prince Georg...,2018,"Feb 8 - Sep 27, 2018",Pending Contribution to 3DEP,Complete
1,gage id01650500,2008 Montgomery County Lidar,2008,"Feb 17, Mar 3, 5, 6, 11, 20, 24, 26, 29, Apr ...",No,Complete
2,gage id01650500,2021 MNCPPC Lidar: Montgomery and Prince Georg...,2021,2021,Pending Contribution to 3DEP,Complete
3,gage id01650500,2013 Montgomery County Maryland National Park ...,2014,"Dec 27 - 28, 2013, Jan 7, 2014",No,Complete
4,gage id 01651000,2018 MNCPPC Lidar: Montgomery and Prince Georg...,2018,"Feb 8 - Sep 27, 2018",Pending Contribution to 3DEP,Complete
5,gage id 01651000,2014 USGS Hurricane Sandy Supplemental for Nat...,2014,"Apr 10 - Dec 20, 2014",Yes,Complete
6,gage id 01651000,2021 MNCPPC Lidar: Montgomery and Prince Georg...,2021,2021,Pending Contribution to 3DEP,Complete
7,washington dc,2018 MNCPPC Lidar: Montgomery and Prince Georg...,2018,"Feb 8 - Sep 27, 2018",Pending Contribution to 3DEP,Complete
8,washington dc,"2022 USGS Northern Virginia (Fairfax, Loudoun,...",2022,Dec. 2022,Expected to meet,In Progress
9,washington dc,2008 Montgomery County Lidar,2008,"Feb 17, Mar 3, 5, 6, 11, 20, 24, 26, 29, Apr ...",No,Complete


EPT files and/or lidar point clouds are stored in their native projection system. In the collections, geodataframe. The `ept_gdf` column stores the location geodataframe transformed to the native EPT CRS. 

In [ ]:
# collections[['Title', 'collectionyear', 'meets3dep', 'Status', 'ept_usiaei', 'ept_noaa', 'ept', 'ept_crs', 'ept_gdf']]

# Map and explore lidar collections 

In [10]:
# convert gdfs to geojson
locations_geojson = leafmap.gdf_to_geojson(locations, epsg=4326)
collections_geojson = leafmap.gdf_to_geojson(collections, epsg=4326)

In [11]:
m = leafmap.Map()
style = {'fillOpacity': 0.5}
m.add_geojson(locations_geojson, layer_name='locations', color='blue', style=style)
m.add_geojson(collections_geojson, layer_name='collections', style=style)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Enter the index number to extract `ept` url path and `ept_bbox` bounding box to retrieve the lidar point clouds using PDAL

In [12]:
index = 0
collection = collections.iloc[[index]]
collection[['Title', 'name']]

In [13]:
count, arrays, metadata, demo_las = lapis.fetch_lidar(collection, geometry_method='polygon', write=True)

In [14]:
array = arrays[0] # 0 is for data and 1 is for datatype + column names

print('no.of points / count: ', count)
print('no. arrays: ', len(arrays))
# print('metadata: ', metadata)

In [15]:
timestamps = gps_time.LidarTimestamps(arrays[0]).get_timestamps()

In [16]:
print(timestamps, collection.collectiondate)

# Lidar visualization (experimental -- may not work)

In [ ]:
# las = leafmap.read_lidar(demo_las)

In [ ]:
# leafmap.view_lidar(demo_las, cmap='terrain', backend='pyvista')

In [ ]:
# leafmap.view_lidar(demo_las, cmap='terrain', backend='ipygany')

In [ ]:
# leafmap.view_lidar(demo_las, cmap='terrain', backend='panel')